#install hazm

In [ ]:
pip install hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.8 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=cde837c3896c50bf05cda1c71492f69e1a7abc175eac0dcfb844daed058a57ef
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
   

In [ ]:
!pip install parsivar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 15.9 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.15-cp311-cp311-linux_x86_64.whl size=4397851 sha256=5b80aaa38b784c952f8be7ada8b1eb2d9b014e20cf0e929ff2a2675343be06b1
  Stored in directory: /root/.cache/pip/wheels/34/62/22/63039c8c4c8da7ff68e81b8357e64c199faff9df9a2b5e5e3b
Successfully built llama-cpp-python


#imports

In [ ]:
from hazm import Normalizer, word_tokenize
import pandas as pd
import re

In [ ]:
from parsivar import Normalizer, SpellCheck

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import time
import csv

#Texts

In [ ]:
# Raw Farsi sentences (you can add your 20 samples here)
texts = [
    "مي خواهم كتاب را بخوانم",
    "سلام دكتر. فشار خونم خيلي ًبالاست !",
    "دارو را   روزي سه بر بخور",
    "از ديروز سر درد دارم 不客气 و هيچ دارويي نخوردم",
    "چك كردم ولي قند خون نرمال بود",
    "سلام دکتر 😊 دارو رو روزی 3 بار @ صبح‌ها بخور، اوکی؟ ❤️ #ممنون",
]

In [ ]:
the_texts = [
    "برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی",
    "آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.",
    "خانم عسگری به دلیل آلرسی فصلی باید روزی دو عدد قرص سیتریسین همراه غذا مصرف کند و شب‌ها  شربت دیفن‌هیدراین قبل از خواب یک قاشق میل نماید.",
    "کودک هفت‌ساله‌ای که دچار عفونت گلو شده است باید هر 8 ساعت 5 شربت فالکسین مصرف کند سی‌سی و هر 6 ساعت یک قاشق شربت استامینوفن برای تب تجویزه شده است.",
    "آقای حسام برای زفلاکس معده باید شب‌هاقرص واموتیدین 40 قبل از خواب  یک عدد و همراه ناهار یک قرص رانیتیدین مصرف کند.",
    "جهت درمان سرماخوردگی، خانم آقاجانی باید روزی یه بار لوراتادین قرص پس از غذا مصرف کند و شب‌ها قبل از خواب یک قاشق شربت دیفندرامین نیز میل نماید.",
    "خانم فرمند با تشخیص برونخیت باید روزی یک عدد قرص آزیترومایسین با معده حالی و هر 80 ساعت یک عدد استامینوفن 500 مصرف نماید قرص .",
    "روزی دو باربرای درد مواصل، آقای تقوی باید قرص ایبوپروفن 40 را هر 8 ساعت بعد از غذا و نابروکسن 250 را  همراه یک لیوان آب میل کند.",
    "کودک ده‌ساله‌ای که کم‌خونی دارد باید روزی یک قرص آن همراه صبحانه و شربت آسکوربیک اسید نیز روزی دو بار برای جذب بهتر .",
    "خانم مرادی که دچار حساسیت پوستی است باید روزی یک عدد قرص لوراتادین صبح‌ها و شب‌ها پماد هیدروکورنیزون را روی نواحی آسیب‌ بالد.",
    "برای درمان عفونت مجازی ادراری، آقای خلیلی باید هر 112 ساعت یک کپسول سفالسین 500 و روزی سه بار قرص مترونیداول همراه غذا .",
    "خانم داوری که با اسال شدید و التهاب  باید روزی دو بار قرص لوپرامید و هر روز صبح یک عدد قرص دگزامتازون . مصرف مایات فراموش شود.",
    "کودک سه‌ساله با گلودرد ویروسی باید هر 6 ساعت یک قاشق شربت استامینوفن و هر 12 ساعت یک قاشق شربت هیستامین بعد از غذا .",
    "آقای رفیعی با آلرژی تنفسی باید روزی یک عدد قرص فکسوونادین با شکم خالی و شب‌ها قبل از خواب شربت دیفن‌هیدرامین .",
    "خانم اجباری که دچار کمبود ویتامین C شده است باید شربت آسکوربیک سید را روزی دو بار بعد از غذا و قرص ژینک را همراه وعده شام مصرف کند.",
    "برای بهبود خواب، آقای شایان باید هر شب قبل از خواب یک عدد قرص ظولپیدم 10 مصرف کند و درصورت اضطراب، صبح‌ها یک قرص الپرازولان با معده خالی میل نماید.",
    "کودک هشت‌ساله با اورتیت میانی باید هر 8 ساعت 5 سی‌سی شربت سفیکسلیم و شب‌ها قبل از خواب یک قاشق شربت دیفن‌هیدرامین  شود.",
    "خانم نظری برای رفلاکس باید صبح‌ها ناشتا یک عدد قرص امپرازول و شب‌ها بعد از شام یک قرص فاموتیدین .",
    "خانم نادری با سردرد مرمن باید روزی هر 66 ساعت یک عدد قرص استامینوفن 5000 مصرف کند  در صورت شدت درد، پماد موضعی منتول روی شقیشقه‌ها.",
    " برای درمان عفونت شدید، آقای پرویزی باید روزی یک عدد قرص آسیترومایسین و هر 12 ساعت یک عدد آمپول چنتامایسین تزریقی طبق دستور زشک بزند."
  ]

#cleaning

In [ ]:
def clean_symbols_and_emojis(text):

    # Define allowed punctuation: Persian and some general
    allowed_punct = "،.؟؛:!"

    # Build regex pattern:
    # Pattern to match any character NOT in:
    # - Persian Unicode range \u0600-\u06FF
    # - English letters a-zA-Z
    # - Digits 0-9 and Persian digits \u06F0-\u06F9
    # - Half-space \u200c
    # - Whitespace \s
    # - Allowed punctuation
    pattern = fr"[^a-zA-Z0-9\u06F0-\u06F9\u0600-\u06FF\u200c\s{re.escape(allowed_punct)}]"

    # Remove disallowed characters (emojis, symbols, etc.)
    cleaned = re.sub(pattern, "", text)

    return cleaned


#Normalizing

In [ ]:
# Initialize hazm normalizer
normalizer = Normalizer()

def normalize_farsi_sentences(sentences):
    """
    Normalize a list of Farsi sentences using hazm.
    """
    normalized = []
    for sentence in sentences:
        cleaned = normalizer.normalize(sentence)
        normalized.append(cleaned)
    return normalized


#applying cleaning and normalizing

In [ ]:
# Clean + Normalize pipeline
cleaned_texts = [clean_symbols_and_emojis(t) for t in the_texts]
normalized_texts = [normalizer.normalize(t) for t in cleaned_texts]

# Show original, cleaned, and normalized
for i, (orig, clean, norm) in enumerate(zip(the_texts, cleaned_texts, normalized_texts), 1):
    print(f"{i}. Original   : {orig}")
    print(f"   Cleaned    : {clean}")
    print(f"   Normalized : {norm}")
    print("-" * 50)

1. Original   : برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی
   Cleaned    : برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی
   Normalized : برای درمان تب و بدن‌درد ، باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود . خانم کرمی
--------------------------------------------------
2. Original   : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.
   Cleaned    : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.
   Normalized : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینو

#creat a csv

In [ ]:
# Create DataFrame
df = pd.DataFrame({
    "Original": the_texts,
    "Cleaned": cleaned_texts,
    "Normalized": normalized_texts
})
df.to_csv("farsi_cleaned_normalized.csv", index=False, encoding="utf-8-sig")
print("✅ Exported to farsi_cleaned_normalized.csv")

✅ Exported to farsi_cleaned_normalized.csv


# loading and testing model

In [ ]:
# توصیه: GPU runtime
# نصب نسخه‌های سازگار
!pip install -U --no-cache-dir "numpy==1.26.4" "pandas==2.2.2" "transformers==4.44.2" "huggingface_hub==0.24.6" gdown -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 318.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 399.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you ha

In [ ]:
!git clone https://github.com/Dadmatech/Nevise.git
%cd Nevise
%pip install -r requirements.txt -q

# دانلود فایل‌های مدل داخل پوشه model/
!mkdir -p model
# ⬇️ این دو لینک را با IDهای واقعی از README عوض کن
!gdown "https://drive.google.com/uc?id=<VOCAB_FILE_ID>"   -O model/persian_vocab.pkl
!gdown "https://drive.google.com/uc?id=<WEIGHTS_FILE_ID>" -O model/nevise_weights.pt


Cloning into 'Nevise'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 39 (delta 19), reused 12 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 14.90 KiB | 462.00 KiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/Nevise
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 23.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1,

In [ ]:
import os, glob, re, torch, pandas as pd
from pathlib import Path

# واردنِ ماژول‌های Nevise
from helpers import bert_tokenize, untokenize_without_unks, load_vocab_dict
from models import SubwordBert

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# پیدا کردن فایل‌های مدل/واژگان
VOCAB_PATHS = glob.glob("model/*.pkl")
CKPT_PATHS  = glob.glob("model/*.pt") + glob.glob("model/*.pth") + glob.glob("model/*.bin")
assert VOCAB_PATHS, "❌ فایل vocab (.pkl) داخل model/ پیدا نشد."
assert CKPT_PATHS,  "❌ فایل وزن مدل (.pt/.pth/.bin) داخل model/ پیدا نشد."

vocab = load_vocab_dict(VOCAB_PATHS[0])
padding_idx = vocab["token2idx"][vocab["pad_token"]]
output_dim  = len(vocab["token2idx"])

model = SubwordBert(screp_dim=0, padding_idx=padding_idx, output_dim=output_dim).to(DEVICE)
state = torch.load(CKPT_PATHS[0], map_location=DEVICE)
model.load_state_dict(state, strict=False)
model.eval()

# (اختیاری) سپر ایمنی: اعداد/دوز/«هر X ساعت» تغییر نکنند
P2E = str.maketrans("۰۱۲۳۴۵۶۷۸۹","0123456789")
A2E = str.maketrans("٠١٢٣٤٥٦٧٨٩","0123456789")
def to_en_digits(s): return s.translate(P2E).translate(A2E)
def safe_numbers(orig: str, gen: str) -> str:
    o, g = to_en_digits(orig), to_en_digits(gen)
    # "هر X ساعت"
    g = re.sub(r"هر\s+\d+\s+ساعت", lambda m: re.search(r"هر\s+\d+\s+ساعت", o).group(0) if re.search(r"هر\s+\d+\s+ساعت", o) else m.group(0), g, count=1)
    # mg / میلی‌گرم
    mg_o = re.search(r"(\d+)\s*(mg|میلی‌ ?گرم)", o)
    if mg_o:
        g = re.sub(r"\d+\s*(?:mg|میلی‌ ?گرم)", mg_o.group(0), g, count=1)
    return g

def nevise_correct_batch(texts, batch_size=16, keep_sensitive=True):
    out = []
    for i in range(0, len(texts), batch_size):
        batch = [t if isinstance(t, str) else "" for t in texts[i:i+batch_size]]
        batch_sentences, batch_bert_dict, batch_splits = bert_tokenize(batch)
        for k in batch_bert_dict:
            batch_bert_dict[k] = batch_bert_dict[k].to(DEVICE)
        # target ساختگی برای forward
        max_words = max((len(s.split()) for s in batch_sentences), default=1)
        dummy_targets = torch.full((len(batch_sentences), max_words), fill_value=padding_idx, dtype=torch.long, device=DEVICE)

        with torch.no_grad():
            _, preds = model(batch_bert_dict, batch_splits, targets=dummy_targets, topk=1)

        lengths = [len(s.split()) for s in batch_sentences]
        decoded = untokenize_without_unks(preds, lengths, vocab, batch_sentences, backoff="pass-through")

        if keep_sensitive:
            decoded = [safe_numbers(src, gen) for src, gen in zip(batch, decoded)]
        out.extend(decoded)
    return out

# --- مسیر CSV اصلی (برگرد یک پوشه عقب اگر فایل بیرون Nevise است) ---
csv_in = Path("../farsi_cleaned_normalized.csv") if not Path("farsi_cleaned_normalized.csv").exists() else Path("farsi_cleaned_normalized.csv")
df = pd.read_csv(csv_in)
df["Normalized"] = df["Normalized"].fillna("").astype(str)

# اجرا فقط روی ستون Normalized
df["Nevise_on_Normalized"] = nevise_correct_batch(df["Normalized"].tolist(), batch_size=8, keep_sensitive=True)

# ذخیره خروجی کنار فایل ورودی
out_path = (csv_in.parent / "farsi_nevise_corrected_on_normalized.csv").as_posix()
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ Saved:", out_path)


In [ ]:
!pip uninstall -y numpy pandas

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2


In [ ]:
!pip install -U --no-cache-dir "numpy==1.26.4" "pandas==2.2.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 318.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 42.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install -U --no-cache-dir "transformers==4.44.2" "huggingface_hub==0.24.6" "dadmatools[full]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dadmatools[full] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 250.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is still looking at multiple versions of dadmatools[full] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 352.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 243.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 82.4 MB/s eta 

In [ ]:
from huggingface_hub import snapshot_download
_ = snapshot_download("xlm-roberta-base")  # کش محلی


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/445 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

(…)d8fbd284dd8127a732ddcf2145b0fc9413b30d40:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

(…)55b47b351a86fe53e6802774d87696bcbc465992:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

(…)aefd190ef844ad994ac74820d9532637bd87b3f4:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"   # بعد از snapshot_download، آفلاین کن

In [ ]:
import pandas as pd
import dadmatools.pipeline.language as language

nlp = language.Pipeline('tok,spellchecker')

def dm_spell_correct(text: str) -> str:
    if not isinstance(text, str):
        text = ""
    doc = nlp(text)
    # بعضی نسخه‌ها خروجی اصلاح‌شده را روی doc._.spellchecker_text می‌گذارند
    corrected = getattr(getattr(doc, "_", None), "spellchecker_text", None)
    return corrected if corrected else str(doc)

df = pd.read_csv("farsi_cleaned_normalized.csv")
df["Normalized"] = df["Normalized"].fillna("").astype(str)
df["DadmaTools_Spell_on_Normalized"] = df["Normalized"].apply(dm_spell_correct)
df.to_csv("farsi_dadmatools_spell_on_normalized.csv", index=False, encoding="utf-8-sig")
print("✅ Saved: farsi_dadmatools_spell_on_normalized.csv")

ValueError: mutable default <class 'dadmatools.pipeline.adapter_transformers.adapter_config.InvertibleAdapterConfig'> for field invertible_adapter is not allowed: use default_factory

In [ ]:
!pip install -U --no-cache-dir bitsandbytes==0.43.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 203.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 210.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 201.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 200.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 179.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 156.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 166.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 259.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nv

In [ ]:
from huggingface_hub import login
login()  # توکن HF رو وارد کن

In [ ]:
import json, torch, pandas as pd, re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# --- Model IDs ---
BASE_ID = "meta-llama/Llama-3.1-8B-Instruct"           # مدل پایه
LORA_ID = "mfsadi/Llama-3.1-8B-spelling-fa"            # LoRA اصلاح املایی

# --- Quantization for VRAM saving (remove if you have plenty of VRAM) ---
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
                             bnb_4bit_compute_dtype=torch.float16)

tok = AutoTokenizer.from_pretrained(BASE_ID)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_cfg,
)
model = PeftModel.from_pretrained(base, LORA_ID)
model.eval()

# --- Prompt مطابق کارت مدل (خروجی JSON با کلید correction) ---
SPELL_PROMPT = (
    "You are tasked with correcting spelling mistakes in the queries that users submitted to a Persian marketplace.\n"
    "Output the corrected query in the following JSON format:\n"
    '- If the input requires correction, use:\n{"correction": "<corrected version of the query>"}\n'
    '- If the input is correct, use:\n{"correction": ""}\n'
    'Here are some examples:\n'
    'query: "ندل چسبی زنانه"  -> {"correction": "صندل چسبی زنانه"}\n'
    'query: "صندلی بادی"      -> {"correction": ""}\n'
)

def build_prompt(query: str) -> str:
    return f"### Human: {SPELL_PROMPT} query: {query}\n ### Assistant:"

# --- Robust JSON extractor (همون ایده‌ی کارت مدل) ---
def extract_correction(text: str):
    try:
        decoder = json.JSONDecoder()
        pos = 0
        while pos < len(text):
            i = text.find('{"correction":', pos)
            if i == -1:
                break
            try:
                obj, j = decoder.raw_decode(text[i:])
                corr = obj.get("correction")
                if corr is not None:
                    return corr
                pos = i + j
            except json.JSONDecodeError:
                pos = i + 1
        return None
    except Exception:
        return None

# --- (اختیاری) سپر ایمنی برای تغییر ندادن اعداد/دوز/ساعت ---
P2E = str.maketrans("۰۱۲۳۴۵۶۷۸۹","0123456789")
A2E = str.maketrans("٠١٢٣٤٥٦٧٨٩","0123456789")
def to_en_digits(s): return s.translate(P2E).translate(A2E)

def safe_numbers(orig: str, gen: str) -> str:
    o, g = to_en_digits(orig), to_en_digits(gen)
    # هر X ساعت
    for m in re.finditer(r"(هر\s+)(\d+)(\s+ساعت)", o):
        g = re.sub(r"هر\s+\d+\s+ساعت", "".join(m.groups()), g, count=1)
    # mg / میلی‌گرم
    for m in re.finditer(r"(\d+)\s*(mg|میلی‌ ?گرم)", o):
        g = re.sub(r"\d+\s*(?:mg|میلی‌ ?گرم)", "".join(m.groups()), g, count=1)
    return g

# --- Batch generate (محافظه‌کارانه) ---
def correct_batch(queries, max_src_len=256, max_new_tokens=80):
    prompts = [build_prompt(q) for q in queries]
    enc = tok(prompts, return_tensors="pt", padding=True, truncation=True, max_length=max_src_len).to(model.device)
    prompt_lens = (enc["input_ids"] != tok.pad_token_id).sum(dim=1)

    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,         # خروجی باثبات
            num_beams=2,
            repetition_penalty=1.1,
            length_penalty=0.0,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
            return_dict_in_generate=True,
        )

    seqs = out.sequences
    texts = []
    for i, seq in enumerate(seqs):
        gen_ids = seq[prompt_lens[i]:]      # فقط بخش تولیدی
        texts.append(tok.decode(gen_ids, skip_special_tokens=True))
    # استخراج correction
    corrs = []
    for q, t in zip(queries, texts):
        c = extract_correction(t)
        if c is None:
            c = q
        elif c == "":                       # اگر گفته نیازی به اصلاح نیست
            c = q
        # سپر ایمنی اعداد/دوز/ساعت
        c = safe_numbers(q, c)
        corrs.append(c)
    return corrs

# --- Run on Normalized column ---
df = pd.read_csv("farsi_cleaned_normalized.csv")
df["Normalized"] = df["Normalized"].fillna("").astype(str)
batch_size = 4
outs = []
norms = df["Normalized"].tolist()
for i in range(0, len(norms), batch_size):
    outs.extend(correct_batch(norms[i:i+batch_size]))

df["llama_spelling_on_Normalized"] = outs
df.to_csv("farsi_llama_spelling_on_normalized.csv", index=False, encoding="utf-8-sig")
print("✅ Saved: farsi_llama_spelling_on_normalized.csv")

PackageNotFoundError: No package metadata was found for bitsandbytes